In [ ]:
import math
import itertools
import numpy as np

import hoomd
from hoomd import md
import gsd.hoomd

In [2]:
from MyCustomForce import MyCustomForce
from hmcmetad import hMCMetaD

In [3]:
frame = gsd.hoomd.Frame()
frame.particles.types = ['A']
frame.particles.N = 1
frame.particles.typeid = [0]
frame.particles.position = [[1,0,0]]
L = 5
frame.configuration.box = [L, L, 0, 0, 0, 0]

with gsd.hoomd.open('./data/lattice.gsd', mode='w') as f:
    f.append(frame)

In [4]:
cpu = hoomd.device.CPU(notice_level=2)
seed = np.random.randint(np.random.randint(10000))

sim = hoomd.Simulation(device=cpu, seed=seed)
sim.create_state_from_gsd(filename='./data/lattice.gsd')

dt = 0.001
kT = 0.2
custom = MyCustomForce()
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                              kT=kT)

nl = md.nlist.Cell(buffer=0.4)

thermostats = md.methods.Brownian(filter=hoomd.filter.All(), 
                                    kT=kT)

thermostats.tally_reservoir_energy = True
thermostats.gamma.default = 1.0
thermostats.gamma_r.default = [1.0, 1.0, 1.0]

integrator = md.Integrator(dt=dt, 
                                methods=[thermostats],
                                forces=[custom])

thermodynamic_properties = md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(thermodynamic_properties)

gsd_writer = hoomd.write.GSD(filename=f'./data/biased_sim_traj.gsd',
                            trigger=hoomd.trigger.Periodic(100),
                            logger=logger,
                            mode='wb')

sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

In [ ]:
init_height = 0.1
mc_stride = 10
metad_stride = 1000
sigma = 0.08
gamma = 20

hmcmetad_action = hMCMetaD(
    sim=sim,
    colvar_mode='double_well_cv',
    colvar_params={},
    kT=kT,
    init_height=init_height,
    mc_stride=mc_stride,
    metad_stride=metad_stride,
    sigma=sigma,
    gamma=gamma,
    cv_min=-2.5,
    cv_max=2.5,
    bins=30,
    seed=seed
)

history_fn = './data/WTMetaD_his.txt'
data_fn = './data/WTMetaD_log.txt'

hmcmetad_action.run(2e6, history_fn=history_fn, data_fn=data_fn)